In [2]:
!pip install ipywidgets

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_thunder-0.2.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/nvfuser-0.2.23a0+6627725-py3.12-linux-x86_

In [3]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

class DataFrameComparer:
    """
    A memory-efficient UI for comparing two pandas DataFrames side by side
    with NVIDIA-themed styling, optimized for large datasets.
    """
    
    def __init__(self, df1, df2, name1="DataFrame 1", name2="DataFrame 2", id_column="id"):
        """
        Initialize with two pandas DataFrames to compare
        
        Parameters:
        -----------
        df1 : pandas.DataFrame
            The first DataFrame to display
        df2 : pandas.DataFrame
            The second DataFrame to display
        name1 : str
            Label for the first DataFrame
        name2 : str
            Label for the second DataFrame
        id_column : str
            Name of the ID column to use for row selection (default: "id")
        """
        # Store only references, not copies
        self.df1 = df1
        self.df2 = df2
        self.name1 = name1
        self.name2 = name2
        self.id_column = id_column
        
        # Validate that the ID column exists in both DataFrames
        if self.id_column not in self.df1.columns:
            raise ValueError(f"ID column '{self.id_column}' not found in first DataFrame")
        if self.id_column not in self.df2.columns:
            raise ValueError(f"ID column '{self.id_column}' not found in second DataFrame")
        
        # For efficiency with large datasets, create a set of IDs for faster lookup
        # and only get a sample for the dropdown to avoid memory issues
        self.df1_id_set = set(self.df1[self.id_column])
        self.df2_id_set = set(self.df2[self.id_column])
        
        # For the dropdown, we'll use a sample of IDs that exist in either dataframe
        # Prioritize IDs that exist in both dataframes
        common_ids = list(self.df1_id_set.intersection(self.df2_id_set))
        df1_only_ids = list(self.df1_id_set - self.df2_id_set)
        df2_only_ids = list(self.df2_id_set - self.df1_id_set)
        
        # Take a sample of at most 1000 IDs for the dropdown to prevent memory issues
        max_sample_size = 1000
        common_sample_size = min(len(common_ids), max_sample_size // 2)
        remaining_slots = max_sample_size - common_sample_size
        
        # Prioritize common IDs, then distribute remaining slots
        if common_sample_size > 0:
            # Sort before slicing to ensure deterministic selection
            common_ids.sort()
            id_sample = common_ids[:common_sample_size]
        else:
            id_sample = []
            
        # Distribute remaining slots between df1_only and df2_only IDs
        if remaining_slots > 0 and (df1_only_ids or df2_only_ids):
            df1_sample_size = min(len(df1_only_ids), remaining_slots // 2)
            df2_sample_size = min(len(df2_only_ids), remaining_slots - df1_sample_size)
            
            if df1_sample_size > 0:
                df1_only_ids.sort()
                id_sample.extend(df1_only_ids[:df1_sample_size])
            
            if df2_sample_size > 0:
                df2_only_ids.sort()
                id_sample.extend(df2_only_ids[:df2_sample_size])
        
        id_sample.sort()
        self.id_sample = id_sample
        
        self._create_widgets()
        
    def _create_widgets(self):
        """Create widgets with NVIDIA-themed styling"""
        # Get all column names from both DataFrames (this doesn't create a copy)
        self.columns1 = list(self.df1.columns)
        self.columns2 = list(self.df2.columns)
        
        # Custom widget styling
        dropdown_layout = widgets.Layout(width='200px')
        
        # ID selection dropdown - uses sampled IDs to prevent memory issues
        id_options = [('All IDs', 'all'), ('Enter ID...', 'enter')]
        id_options.extend([(str(id_val), id_val) for id_val in self.id_sample])
        
        self.select_id = widgets.Dropdown(
            options=id_options,
            value='all',
            description='Select ID:',
            style={'description_width': 'initial'},
            layout=dropdown_layout
        )
        
        # Text input for entering a specific ID
        self.id_input = widgets.Text(
            value='',
            placeholder='Type an ID value',
            description='ID:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px', display='none')
        )
        
        # Column selection for DataFrame 1
        self.select_cols1 = widgets.Dropdown(
            options=[('All Columns', 'all')] + [(col, col) for col in self.columns1],
            value='all',
            description=f'{self.name1}:',
            style={'description_width': 'initial'},
            layout=dropdown_layout
        )
        
        # Column selection for DataFrame 2
        self.select_cols2 = widgets.Dropdown(
            options=[('All Columns', 'all')] + [(col, col) for col in self.columns2],
            value='all',
            description=f'{self.name2}:',
            style={'description_width': 'initial'},
            layout=dropdown_layout
        )
        
        # Page controls for navigating large datasets
        self.page_size = widgets.Dropdown(
            options=[5, 10, 20, 50],
            value=10,
            description='Page Size:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='150px')
        )
        
        self.page_num = widgets.BoundedIntText(
            value=1,
            min=1,
            max=9999,
            step=1,
            description='Page:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='120px')
        )
        
        # Apply button with NVIDIA styling
        self.apply_button = widgets.Button(
            description='Compare',
            disabled=False,
            button_style='success',
            tooltip='Compare DataFrames',
            layout=widgets.Layout(width='120px')
        )
        
        # Output area for the DataFrames
        self.output = widgets.Output()
        
        # Connect event handlers
        self.apply_button.on_click(self._handle_compare)
        self.select_id.observe(self._handle_id_selection_change, names='value')
        
        # Layout with NVIDIA styling
        title = widgets.HTML(
            "<div style='background-color: #1a1a1a; padding: 12px; " 
            "border-radius: 8px; margin-bottom: 15px; font-family: \"Segoe UI\", sans-serif;'>"
            "<h4 style='margin: 0; color: #76B900; font-weight: 500;'>DataFrame Comparison</h4></div>"
        )
        
        # Row for ID selection
        id_row = widgets.HBox([
            self.select_id,
            self.id_input
        ], layout=widgets.Layout(justify_content='flex-start', margin='10px 0'))
        
        # Row for column selections
        col_row = widgets.HBox([
            self.select_cols1, 
            self.select_cols2
        ], layout=widgets.Layout(justify_content='space-around', margin='10px 0'))
        
        # Row for pagination controls
        page_row = widgets.HBox([
            self.page_size,
            self.page_num,
            self.apply_button
        ], layout=widgets.Layout(justify_content='flex-start', margin='0 0 15px 0'))
        
        self.main_layout = widgets.VBox([
            title,
            id_row,
            col_row, 
            page_row,
            self.output
        ], layout=widgets.Layout(margin='10px'))
    
    def _handle_id_selection_change(self, change):
        """Handle changes to the ID selection dropdown"""
        if change['new'] == 'enter':
            self.id_input.layout.display = 'block'
        else:
            self.id_input.layout.display = 'none'
    
    def _render_table_html(self, df_subset, name, has_data=True):
        """Render a single table with NVIDIA styling"""
        if not has_data:
            return f"""
            <div style="background-color: #1a1a1a; padding: 10px 15px; border-radius: 6px; margin-bottom: 10px; border-left: 4px solid #76B900;">
                <p style="margin: 0; font-weight: 500; color: #76B900;">{name}</p>
            </div>
            <div style="border-radius: 6px; padding: 20px; text-align: center; background-color: #2d2d2d; color: #aaa;">
                No matching data found
            </div>
            """
        
        # Use to_html with memory-efficient settings to generate the table HTML
        # minimal=True reduces HTML output size
        html_table = df_subset.to_html(
            index=True, 
            classes="dataframe nvidia-table",
            na_rep="",  # Better for memory
            float_format=lambda x: f"{x:.3f}",  # Format floats to reduce size
            border=0,
            justify='left'
        )
        
        return f"""
        <div style="background-color: #1a1a1a; padding: 10px 15px; border-radius: 6px; margin-bottom: 10px; border-left: 4px solid #76B900;">
            <p style="margin: 0; font-weight: 500; color: #76B900;">{name}</p>
        </div>
        <div style="border-radius: 6px; overflow: hidden; box-shadow: 0 3px 6px rgba(0,0,0,0.2);">
            {html_table}
        </div>
        """
    
    def _handle_compare(self, _):
        """Compare button click handler optimized for memory efficiency"""
        with self.output:
            self.output.clear_output()
            
            # Determine which columns to display - don't create copies here
            if self.select_cols1.value == 'all':
                cols1 = self.columns1
            else:
                cols1 = [self.select_cols1.value]
                
            if self.select_cols2.value == 'all':
                cols2 = self.columns2
            else:
                cols2 = [self.select_cols2.value]
            
            # Get the page size and current page
            page_size = self.page_size.value
            page = self.page_num.value
            
            # Memory-efficient filtering for ID or paged data
            if self.select_id.value == 'enter':
                # Get ID from text input
                try:
                    id_value = type(next(iter(self.df1_id_set)))(self.id_input.value)
                except (ValueError, StopIteration):
                    display(HTML(f"<div style='color: #76B900; padding: 10px; background-color: #1a1a1a; border-radius: 4px;'>Invalid ID format: {self.id_input.value}</div>"))
                    return
                    
                # Check if ID exists in either DataFrame
                in_df1 = id_value in self.df1_id_set
                in_df2 = id_value in self.df2_id_set
                
                if not in_df1 and not in_df2:
                    display(HTML(f"<div style='color: #76B900; padding: 10px; background-color: #1a1a1a; border-radius: 4px;'>ID {id_value} not found in either DataFrame</div>"))
                    return
                
                # Memory-efficient filtering
                if in_df1:
                    # Use .loc for more efficient row selection without copying the entire DataFrame
                    mask1 = self.df1[self.id_column] == id_value
                    idx1 = mask1.idxmax() if mask1.any() else None
                    df1_display = self.df1.loc[[idx1], cols1] if idx1 is not None else pd.DataFrame(columns=cols1)
                else:
                    df1_display = pd.DataFrame(columns=cols1)
                
                if in_df2:
                    # Use .loc for more efficient row selection without copying the entire DataFrame
                    mask2 = self.df2[self.id_column] == id_value
                    idx2 = mask2.idxmax() if mask2.any() else None
                    df2_display = self.df2.loc[[idx2], cols2] if idx2 is not None else pd.DataFrame(columns=cols2)
                else:
                    df2_display = pd.DataFrame(columns=cols2)
                
                # Status messages
                id_status_html = ""
                if not in_df1:
                    id_status_html += f"<div style='color: #76B900; padding: 5px; background-color: #1a1a1a; border-radius: 4px;'>Note: ID {id_value} not found in {self.name1}</div>"
                if not in_df2:
                    id_status_html += f"<div style='color: #76B900; padding: 5px; background-color: #1a1a1a; border-radius: 4px;'>Note: ID {id_value} not found in {self.name2}</div>"
                
            elif self.select_id.value != 'all':
                # Use selected ID from dropdown
                id_value = self.select_id.value
                
                # Check if ID exists in either DataFrame
                in_df1 = id_value in self.df1_id_set
                in_df2 = id_value in self.df2_id_set
                
                # Memory-efficient filtering
                if in_df1:
                    # Use .loc for more efficient row selection without copying the entire DataFrame
                    mask1 = self.df1[self.id_column] == id_value
                    idx1 = mask1.idxmax() if mask1.any() else None
                    df1_display = self.df1.loc[[idx1], cols1] if idx1 is not None else pd.DataFrame(columns=cols1)
                else:
                    df1_display = pd.DataFrame(columns=cols1)
                
                if in_df2:
                    # Use .loc for more efficient row selection without copying the entire DataFrame
                    mask2 = self.df2[self.id_column] == id_value
                    idx2 = mask2.idxmax() if mask2.any() else None
                    df2_display = self.df2.loc[[idx2], cols2] if idx2 is not None else pd.DataFrame(columns=cols2)
                else:
                    df2_display = pd.DataFrame(columns=cols2)
                
                # Status messages
                id_status_html = ""
                if not in_df1:
                    id_status_html += f"<div style='color: #76B900; padding: 5px; background-color: #1a1a1a; border-radius: 4px;'>Note: ID {id_value} not found in {self.name1}</div>"
                if not in_df2:
                    id_status_html += f"<div style='color: #76B900; padding: 5px; background-color: #1a1a1a; border-radius: 4px;'>Note: ID {id_value} not found in {self.name2}</div>"
                
            else:
                # Show paged data - calculate indices for current page
                start_idx = (page - 1) * page_size
                end_idx = start_idx + page_size
                
                # Check if we're past the end of the data
                if start_idx >= len(self.df1):
                    df1_display = pd.DataFrame(columns=cols1)
                    has_df1_data = False
                else:
                    # Use efficient row indexing without copying the entire DataFrame
                    # iloc is memory-efficient for sequential row access
                    end_df1 = min(end_idx, len(self.df1))
                    df1_display = self.df1.iloc[start_idx:end_df1][cols1]
                    has_df1_data = True
                
                if start_idx >= len(self.df2):
                    df2_display = pd.DataFrame(columns=cols2)
                    has_df2_data = False
                else:
                    # Use efficient row indexing without copying the entire DataFrame
                    end_df2 = min(end_idx, len(self.df2))
                    df2_display = self.df2.iloc[start_idx:end_df2][cols2]
                    has_df2_data = True
                
                # Pagination info
                total_pages1 = (len(self.df1) + page_size - 1) // page_size
                total_pages2 = (len(self.df2) + page_size - 1) // page_size
                max_pages = max(total_pages1, total_pages2)
                
                id_status_html = f"""
                <div style='color: #76B900; padding: 5px; background-color: #1a1a1a; border-radius: 4px; margin-top: 10px;'>
                    Page {page} of {max_pages} | Showing rows {start_idx+1}-{end_idx} | 
                    Total: {len(self.df1)} rows in {self.name1}, {len(self.df2)} rows in {self.name2}
                </div>
                """
            
            # Render tables - process each DataFrame independently
            table1_html = self._render_table_html(
                df1_display, 
                self.name1, 
                has_data=(not df1_display.empty)
            )
            
            table2_html = self._render_table_html(
                df2_display, 
                self.name2, 
                has_data=(not df2_display.empty)
            )
            
            # NVIDIA-themed styling for better visual appeal
            html = f"""
            <div style="display: flex; width: 100%; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;">
                <div style="flex: 1; padding-right: 10px;">
                    {table1_html}
                </div>
                <div style="flex: 1; padding-left: 10px;">
                    {table2_html}
                </div>
            </div>
            {id_status_html}
            <style>
            .nvidia-table {{
                border-collapse: collapse;
                width: 100%;
                font-size: 13px;
                border: none;
            }}
            .nvidia-table th {{
                background-color: #2d2d2d;
                color: #76B900;
                padding: 10px 12px;
                text-align: left;
                border: 1px solid #444;
                font-weight: 500;
            }}
            .nvidia-table td {{
                padding: 8px 12px;
                border: 1px solid #444;
                background-color: #1a1a1a;
                color: #f0f0f0;
            }}
            .nvidia-table tr:nth-child(even) td {{
                background-color: #262626;
            }}
            .nvidia-table tr:hover td {{
                background-color: #333;
            }}
            </style>
            """
            display(HTML(html))
    
    def display(self):
        """Display the UI and initial comparison"""
        display(self.main_layout)
        self._handle_compare(None)


# Memory-efficient function to compare by ID
def memory_efficient_compare_by_id(df1, df2, id_value, id_column="id", name1="DataFrame 1", name2="DataFrame 2"):
    """
    Compare a specific row from two DataFrames based on ID value
    with optimizations for memory efficiency
    
    Parameters:
    -----------
    df1 : pandas.DataFrame
        The first DataFrame
    df2 : pandas.DataFrame
        The second DataFrame
    id_value : any
        The ID value to look up
    id_column : str
        The column name containing the ID (default: "id")
    name1 : str
        Label for the first DataFrame
    name2 : str
        Label for the second DataFrame
    """
    # Check if ID exists in either DataFrame
    in_df1 = id_value in set(df1[id_column])
    in_df2 = id_value in set(df2[id_column])
    
    if not in_df1 and not in_df2:
        return HTML(f"<div style='color: #76B900; padding: 10px; background-color: #1a1a1a; border-radius: 4px;'>ID {id_value} not found in either DataFrame</div>")
    
    # Memory-efficient filtering
    if in_df1:
        # Find the index of the row with this ID
        mask1 = df1[id_column] == id_value
        if mask1.any():
            idx1 = mask1.idxmax()
            df1_html = df1.loc[[idx1]].to_html(index=True, classes="dataframe nvidia-table")
        else:
            df1_html = "<p>No data found</p>"
    else:
        df1_html = "<p>ID not found in this DataFrame</p>"
    
    if in_df2:
        # Find the index of the row with this ID
        mask2 = df2[id_column] == id_value
        if mask2.any():
            idx2 = mask2.idxmax()
            df2_html = df2.loc[[idx2]].to_html(index=True, classes="dataframe nvidia-table")
        else:
            df2_html = "<p>No data found</p>"
    else:
        df2_html = "<p>ID not found in this DataFrame</p>"
    
    # Status messages
    id_status_html = ""
    if not in_df1:
        id_status_html += f"<div style='color: #76B900; padding: 5px; background-color: #1a1a1a; border-radius: 4px; margin-top: 10px;'>Note: ID {id_value} not found in {name1}</div>"
    if not in_df2:
        id_status_html += f"<div style='color: #76B900; padding: 5px; background-color: #1a1a1a; border-radius: 4px; margin-top: 10px;'>Note: ID {id_value} not found in {name2}</div>"
    
    # Create the HTML with NVIDIA styling
    html = f"""
    <div style="display: flex; width: 100%; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; margin-top: 15px;">
        <div style="flex: 1; padding-right: 15px;">
            <div style="background-color: #1a1a1a; padding: 10px 15px; border-radius: 8px; margin-bottom: 10px; border-left: 4px solid #76B900;">
                <p style="margin: 0; font-weight: 500; color: #76B900; font-size: 16px;">{name1}</p>
            </div>
            <div style="border-radius: 8px; overflow: hidden; box-shadow: 0 3px 6px rgba(0,0,0,0.2);">
                {df1_html}
            </div>
        </div>
        <div style="flex: 1; padding-left: 15px;">
            <div style="background-color: #1a1a1a; padding: 10px 15px; border-radius: 8px; margin-bottom: 10px; border-left: 4px solid #76B900;">
                <p style="margin: 0; font-weight: 500; color: #76B900; font-size: 16px;">{name2}</p>
            </div>
            <div style="border-radius: 8px; overflow: hidden; box-shadow: 0 3px 6px rgba(0,0,0,0.2);">
                {df2_html}
            </div>
        </div>
    </div>
    {id_status_html}
    <style>
    .nvidia-table {{
        border-collapse: collapse;
        width: 100%;
        font-size: 13px;
        border: none;
    }}
    .nvidia-table th {{
        background-color: #2d2d2d;
        color: #76B900;
        padding: 10px 12px;
        text-align: left;
        border: 1px solid #444;
        font-weight: 500;
    }}
    .nvidia-table td {{
        padding: 8px 12px;
        border: 1px solid #444;
        background-color: #1a1a1a;
        color: #f0f0f0;
    }}
    .nvidia-table tr:nth-child(even) td {{
        background-color: #262626;
    }}
    .nvidia-table tr:hover td {{
        background-color: #333;
    }}
    </style>
    """
    return HTML(html)


In [ ]:
# Example usage
if __name__ == "__main__":
    # Sample data
    import numpy as np
    
    # Create two sample DataFrames with ID column
    np.random.seed(42)
    
    # First DataFrame with IDs from 1 to 20000
    df1 = pd.DataFrame({
        'id': range(1, 20001),
        'value_A': np.random.randn(20000),
        'category': np.random.choice(['X', 'Y', 'Z'], 20000),
    })
    
    # Second DataFrame with IDs from 1 to 20000 but with slightly different values
    df2 = pd.DataFrame({
        'id': range(1, 20001),
        'value_B': np.random.randn(20000) * 1.2,
        'category': np.random.choice(['X', 'Y', 'Z'], 20000),
    })
    
    # Option 1: Use the memory-efficient UI with ID selection
    comparer = DataFrameComparer(df1, df2, "Original", "Modified", id_column="id")
    comparer.display()
    
    # Option 2: Use the memory-efficient function to compare a specific ID
    # display(memory_efficient_compare_by_id(df1, df2, 5, id_column="id", name1="Original", name2="Modified"))

In [ ]:
# Silence Warnings (HuggingFace internal warnings)

%env PYTHONWARNINGS=ignore
import warnings
warnings.filterwarnings("ignore")
import os

In [2]:
import json
import fasttext
import pandas as pd
import cudf
import dask_cudf
import numpy as np
import cupy as cp
from pathlib import Path
from typing import Optional, Tuple, Any, Dict, List
from huggingface_hub import hf_hub_download

from nemo_curator import get_client
from nemo_curator.classifiers import FineWebNemotronEduClassifier, FineWebMixtralEduClassifier
from nemo_curator.datasets import DocumentDataset
from nemo_curator.utils.distributed_utils import load_object_on_worker
from nemo_curator.utils.distributed_utils import get_device_total_memory

### Initializing NeMo Curator Client
This step initializes the NeMo Curator client to enable distributed classification using GPU-based processing.

In [ ]:
%env CUDA_VISIBLE_DEVICES 3

In [ ]:
client = get_client(cluster_type="gpu")

### Setting Output File Paths
Defines the paths where classification results, threshold values, and final bucketed results will be stored.

# Preparing Text Data for Classification
- We create a sample dataset with diverse topics.
- Optionally, users can provide a directory containing JSONL files for classification.

# Step 1: Run the Classifiers

1. Compute the floating-point classification score for each classifier.

**Note:** Dask operations are lazy, meaning the classifiers won’t execute until an eager operation like `to_json`, `compute`, or `persist` is called.

### FastText Quality Classifier

The **FastText Quality Classifier** uses the [`fasttext-oh-eli5`](https://huggingface.co/mlfoundations/fasttext-oh-eli5) model from Hugging Face to assess text quality. It distinguishes **high-quality** (`__label__hq`) responses from lower-quality ones (`__label__cc`).  

NeMo Curator allows users to define custom modules like this, enabling seamless integration of specialized models.  

- **Model:** [`mlfoundations/fasttext-oh-eli5`](https://huggingface.co/mlfoundations/fasttext-oh-eli5)  
- **Training Data:** Reddit ELI5 vs. Wikipedia (200k examples)  
- **Output:** Confidence score + optional binary classification (where 1 represents high quality text and 0 represents low quality text)  

🔗 **More details:** [Hugging Face Model Card](https://huggingface.co/mlfoundations/fasttext-oh-eli5)

In [ ]:
# Define classifier score mapping
classifier_scores = {
    "nemotron-score": {
        "int_score": "fineweb-nemotron-edu-score-int",
        "float_score": "fineweb-nemotron-edu-score"
    },
    "mixtral-score": {
        "int_score": "fineweb-mixtral-edu-score-int",
        "float_score": "fineweb-mixtral-edu-score"
    },
    "fasttext-score": {
        "int_score": "fasttext-quality-score-int",
        "float_score": "fasttext-quality-score"
    }
}



# Initialize classifiers
classifiers = [
    FineWebNemotronEduClassifier(batch_size=1024,
                         pred_column=classifier_scores["nemotron-score"]["float_score"],
                         int_column=classifier_scores["nemotron-score"]["int_score"]),
    FineWebMixtralEduClassifier(batch_size=1024,
                         pred_column=classifier_scores["mixtral-score"]["float_score"],
                         int_column=classifier_scores["mixtral-score"]["int_score"]),
    FastTextQualityClassifier(pred_column=classifier_scores["fasttext-score"]["float_score"],
                         int_column=classifier_scores["fasttext-score"]["int_score"])
]

In [9]:
client.cluster.close()
client.shutdown()

In [ ]:
output_dataset = input_dataset
for classifier in classifiers:
    output_dataset = classifier(dataset=output_dataset)

# Dropping int columns
# As we add new based on a threshold (in the following columns)
output_dataset = output_dataset.df.drop(columns=[v["int_score"] for v in classifier_scores.values()])
output_dataset.to_parquet(path=OUTPUT_CLASSIFICATION_RESULTS)

In [9]:
del classifiers, output_dataset, input_dataset

### Read Back in the scored Data Frame

In [ ]:
scored_data = DocumentDataset.read_parquet(OUTPUT_CLASSIFICATION_RESULTS, backend="cudf")
scored_data.df.head()

# Step 2: Compute Score Thresholds

### Why Compute Thresholds?
- To categorize classification scores into percentile-based bins.
- Ensures results are comparable across different classifiers.

### Approach:
1. **Extract classifier scores** from the sampled dataset.
2. **Compute weighted percentiles** for each classifier.
3. **Save percentile thresholds** for later use in mapping scores.

> **Note:** The percentile calculation is weighted by token count so that longer texts (with more tokens) have a greater impact on the thresholds. This ensures that the bins accurately reflect the distribution of content, giving a more meaningful categorization of the scores.

In [11]:
def weighted_percentile(data, percentiles, weights):
    """
    Compute weighted percentiles with the "inverted_cdf" method.

    Parameters:
      data : array-like, the data values.
      percentiles : scalar or array-like, percentiles in [0, 100].
      weights : array-like, the weights for each data value.
    
    Returns:
      The weighted percentile values.
    """
    data = np.asarray(data)
    weights = np.asarray(weights)
    
    # Sort data and associated weights
    sorter = np.argsort(data)
    data_sorted = data[sorter]
    weights_sorted = weights[sorter]
    
    # Compute the cumulative sum of weights and normalize it to [0, 1]
    cum_weights = np.cumsum(weights_sorted)
    total_weight = cum_weights[-1]
    normalized_cum_weights = cum_weights / total_weight

    # For each desired percentile, find the first data value where
    # the normalized cumulative weight is >= (percentile / 100).
    percentiles = np.atleast_1d(percentiles)
    results = []
    for p in percentiles:
        # np.searchsorted returns the index where (p/100) should be inserted 
        # to maintain order.
        idx = np.searchsorted(normalized_cum_weights, p / 100.0, side='left')
        results.append(data_sorted[idx])
    
    return np.array(results)


def compute_thresholds(score_ar: np.ndarray, token_ar: np.ndarray) -> Dict[str, float]:
    """
    Compute percentile-based thresholds for a given score column using weighted percentiles.

    Args:
        score_ar (np.ndarray): Array containing the scores.
        token_ar (np.ndarray): Array containing token counts for weighting.

    Returns:
        Dict[str, float]: Dictionary containing percentile thresholds.
    """
    percentiles = np.arange(5, 100, 5)
    # NumPy < 2.0 does not support the "inverted_cdf" method for computing percentiles 
    # with weights directly via np.percentile (see commented-out equivalent code below).
    # To achieve the same result, we manually implement the weighted percentile computation
    # using NumPy primitives.
    # thresholds = np.percentile(cc_df_score, percentiles, weights=cc_df_tokens, method='inverted_cdf')
    thresholds = weighted_percentile(score_ar, percentiles, weights=token_ar)
    return {int(percentile): float(thresh) for percentile, thresh in zip(percentiles, thresholds)}


def compute_thresholds_for_score_columns(
    df: cudf.DataFrame, text_col_name: str, score_col_names: List[str]
) -> Dict[str, Dict[str, float]]:
    """
    Compute percentile-based thresholds for all specified score columns in a DataFrame.

    Args:
        df (cudf.DataFrame): The DataFrame containing the score columns and text column.
        text_col_name (str): The name of the text column used to derive token counts.
        score_col_names (List[str]): List of column names for which thresholds should be computed.

    Returns:
        Dict[str, Dict[str, float]]: A dictionary mapping each score column to its percentile thresholds.
    """
    threshold_dict = {}
    token_series = df[text_col_name].str.byte_count()

    for score_col in score_col_names:
        threshold_dict[score_col] = compute_thresholds(df[score_col].values.get(), token_series.values.get())

    return threshold_dict


def save_thresholds(threshold_dict: Dict[str, Dict[str, float]],  file_name) -> None:
    """
    Save computed thresholds to a JSON file.

    Args:
        threshold_dict (Dict[str, Dict[str, float]]): The dictionary containing computed thresholds.
        file_name (str, optional): The name of the output JSON file. Defaults to "thresholds.json".
    Returns:
        None
    """
    with open(file_name, 'w') as fout:
        json.dump(file_name, fout, indent=4)
    print(f"Thresholds saved to {file_name}")

In [ ]:
# Adjust fraction based on how much can fit in a single GPU (1/2 ish)
gpu_memory_available = get_device_total_memory()/2
frac = max(1, scored_data.df.memory_usage(deep=True).sum().compute()/gpu_memory_available)
sampled_data =  scored_data.df.sample(frac=frac).repartition(npartitions=1)

score_col_names = [v["float_score"] for v in classifier_scores.values()]
threshold_dict = sampled_data.map_partitions(compute_thresholds_for_score_columns, text_col_name="text", score_col_names=score_col_names).compute().iloc[0]
save_thresholds(threshold_dict, OUTPUT_CLASSIFIER_THRESHOLDS)

In [ ]:
threshold_dict

# Step 3: Convert Floating-Point Scores to Integer Scores

### Why Convert?
- Floating-point scores are mapped to integer categories (0-19) for easier comparison.
- Integer scores are computed using **percentile-based thresholds**.

### Process:
1. **Retrieve percentile thresholds** from saved JSON.
2. **Apply the thresholds to map scores to integer bins**.
3. **Store integer scores in the dataset** for final ensemble computation.

In [14]:
def map_scores(df, score_col_name: str, score_int_name: str, bins: List[float]):
    """
    Given a DataFrame df and a column of original scores, 
    use cp.digitize to map them into integer bins using the given thresholds.
    """
    pred_orig_score = cp.array(df[score_col_name])
    pred_int_score = cp.digitize(pred_orig_score, bins)
    df[score_int_name] = pred_int_score
    return df

def map_score_columns(df: cudf.DataFrame, score_col_names: List[str], threshold_dict: Dict[str, dict]):
    """
    For each score column in score_col_names, this function:
      1. Creates a new column name by appending '-int'
      2. Retrieves the corresponding thresholds from threshold_dict,
         sorts them (using the keys which are assumed to be strings of numbers),
      3. Passes the bins to map_scores to create the integer score column.
    """
    for score_col_name in score_col_names:
        # Build the new integer score column name.
        score_int_name = score_col_name + "-int"
        thresholds = threshold_dict.get(score_col_name)
        if thresholds is None:
            raise ValueError(f"No thresholds found for score column '{score_col_name}'")
        
        sorted_keys = sorted(thresholds.keys(), key=lambda x: int(x))
        # Use cp.array to create a CuPy array from the list of threshold values.
        bins = cp.array([thresholds[k] for k in sorted_keys])
        
        # Map the original score column to the new integer score column.
        df = map_scores(df, score_col_name, score_int_name, bins)
    return df


In [ ]:
scored_data.df = scored_data.df.map_partitions(map_score_columns, score_col_names, threshold_dict)
scored_data.head()

# Step 4: Compute the Final Ensembled Score

### Purpose:
- To combine the predictions from multiple classifiers into a **single representative score**.
- The ensemble score is computed as the **maximum of all integer scores** across classifiers.

### Approach:
1. **Extract integer scores from each classifier.**
2. **Compute the max integer score for each data point.**
3. **Store the final ensemble score in the dataset.**

In [16]:
int_column_names = [f'{v["float_score"]}-int' for v in classifier_scores.values()]
scored_data.df['ensemble-max-int'] = scored_data.df[int_column_names].max(axis=1)

In [ ]:
scored_data.df.head()

# Step 5: Write Results to Partitioned Buckets


### Purpose:
- Organize and store classified results in a **structured, partitioned format** to facilitate **annealing-based training** for downstream **LLM fine-tuning** and optimization.

In [ ]:
scored_data.to_parquet(OUTPUT_BUCKETED_RESULTS, partition_on="ensemble-max-int")

# Verify Results

### Process:
1. **List available partitions** (each corresponds to a score bucket).
2. **Read a sample partition** and validate data integrity.

In [ ]:
all_buckets = sorted(os.listdir(OUTPUT_BUCKETED_RESULTS))
print(all_buckets)
first_bucket= DocumentDataset.read_parquet(os.path.join(OUTPUT_BUCKETED_RESULTS, all_buckets[0]))
first_bucket.head()